In [63]:
import cvxpy as cp
import numpy as np

In [52]:
# Data of the problem
cost = np.array([3]).reshape(-1,1)
all_demand = np.array([[1.5],[2],[3.5],[4],[5.5]]).reshape(-1,1)
selling_price = np.array([5]).reshape(-1,1)
salvage_price= np.array([1]).reshape(-1,1)
senario = len(all_demand)
senario_prob = (1/senario)*(np.ones_like(all_demand))
limit = -100000

In [59]:
def solve_second_stage_subproblem(selling_price,salvage_price,supply,demand):

    sell = cp.Variable(shape=(1,1),name="sell")
    salvage = cp.Variable(shape=(1,1),name="salvage")

    objective = cp.Minimize(-1*(selling_price.T@sell)-1*(salvage_price.T@salvage))   # Objective function for second stage problem

    constraints =  [sell+salvage<=supply,sell<=demand,salvage>=0,sell>=0]  # Constriants for second stage problem

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return problem

In [60]:
def solve_first_stage_problem(produce,limit,g_ks,alpha_ks,cost,all_demand):
    
    # Intiatel variables for the problem
    produce  = cp.Variable(shape=(1,1),name="produce")
    v = cp.Variable(shape=(1,1),name="value")

    # Contraints for first stage problem
    constraints = []
    for cut in range(0,len(g_ks)):
        constraints.append(np.array([g_ks[cut]]).T@produce+np.array([alpha_ks[cut]])<=v)
    constraints.extend([produce>=0,v>=-1000000,produce>=limit,produce<=max(all_demand)])

    objective = cp.Minimize((cost.T@produce)+v)    #Objective function for first stage problem

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return problem

In [61]:

# Intitalize g and alpha for storing gs and alphas for each cut 
g_ks = np.array([[]])
alpha_ks = np.array([[]])

i=0
while True:

    # Solve Second stage problem for each demand and store its duals and objective values
    duals = []
    objs= []
    for demand in all_demand:

        second_stage_sol = solve_second_stage_subproblem(selling_price,salvage_price,produce,demand)

        temp_dual = second_stage_sol.constraints[0].dual_value   # Take the duals of 1st contraint
        temp_obj = second_stage_sol.value                        # Take the objective value of second stage problem

        # Store duals and objective values for each senario
        duals.append(temp_dual) 
        objs.append(temp_obj)

    # Reshaping the values 
    duals = np.array(duals).reshape(-1,1)
    objs = np.array(objs).reshape(-1,1)

    # Computing g_ks and alpha_ks 
    g_ks_temp = -senario_prob.T@duals
    alpha_ks_temp = senario_prob.T@objs - g_ks_temp.T@produce
    
    # Store the values in data storage
    g_ks = np.append(g_ks,g_ks_temp)
    alpha_ks = np.append(alpha_ks,alpha_ks_temp)

    # Solve the first stage problem
    first_stage_sol = solve_first_stage_problem(produce,limit,g_ks,alpha_ks,cost,all_demand)
    new_produce = first_stage_sol.var_dict["produce"].value
    new_limit = first_stage_sol.var_dict["value"].value

    produce,limit = new_produce,new_limit # swap the values

    if i==10:
       break 
    i=i+1



-4.42750483617983
-4.32948953948668
-4.26166131328936
-4.199999983099287
-4.199999984194896
-4.199999970902288
-4.19999998054463
-4.199999980034166
-4.199999978559637
-4.199999981047965
-4.199999981682764


[Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Expression(AFFINE, UNKNOWN, (1,))),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, NONPOSITIVE, ())),
 Inequality(Constant(CONSTANT, NONPOSITIVE, (1, 1))),
 Inequality(Variable((1, 1), produce))]